# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 7 2022 1:03PM,250193,10,CTF0011491,"Citieffe, Inc.",Released
1,Nov 7 2022 12:57PM,250192,10,0086148679,ISDIN Corporation,Released
2,Nov 7 2022 12:57PM,250192,10,0086148761,ISDIN Corporation,Released
3,Nov 7 2022 12:57PM,250192,10,0086148824,ISDIN Corporation,Released
4,Nov 7 2022 12:57PM,250192,10,0086148839,ISDIN Corporation,Released
5,Nov 7 2022 12:57PM,250192,10,0086149080,ISDIN Corporation,Released
6,Nov 7 2022 12:57PM,250192,10,0086149081,ISDIN Corporation,Released
7,Nov 7 2022 12:57PM,250192,10,0086149094,ISDIN Corporation,Released
8,Nov 7 2022 12:57PM,250192,10,0086149103,ISDIN Corporation,Released
9,Nov 7 2022 12:57PM,250192,10,0086149114,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,250189,Released,1
34,250190,Released,1
35,250191,Released,1
36,250192,Released,11
37,250193,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250189,NaN,NaN,1.0
250190,NaN,NaN,1.0
250191,NaN,NaN,1.0
250192,NaN,NaN,11.0
250193,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250118,0.0,6.0,0.0
250121,0.0,1.0,0.0
250130,0.0,0.0,1.0
250131,0.0,0.0,1.0
250133,0.0,1.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250118,0,6,0
250121,0,1,0
250130,0,0,1
250131,0,0,1
250133,0,1,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250118,0,6,0
1,250121,0,1,0
2,250130,0,0,1
3,250131,0,0,1
4,250133,0,1,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250118,,6,
1,250121,,1,
2,250130,,,1
3,250131,,,1
4,250133,,1,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 7 2022 1:03PM,250193,10,"Citieffe, Inc."
1,Nov 7 2022 12:57PM,250192,10,ISDIN Corporation
12,Nov 7 2022 12:52PM,250191,22,"NBTY Global, Inc."
13,Nov 7 2022 12:52PM,250190,22,"NBTY Global, Inc."
14,Nov 7 2022 12:52PM,250189,22,"NBTY Global, Inc."
15,Nov 7 2022 12:49PM,250188,10,"Citieffe, Inc."
16,Nov 7 2022 12:45PM,250187,10,Emerginnova
21,Nov 7 2022 12:42PM,250186,10,"Snap Medical Industries, LLC"
39,Nov 7 2022 12:32PM,250182,19,"AdvaGen Pharma, Ltd"
41,Nov 7 2022 12:28PM,250181,19,"Innogenix, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 7 2022 1:03PM,250193,10,"Citieffe, Inc.",,,1
1,Nov 7 2022 12:57PM,250192,10,ISDIN Corporation,,,11
2,Nov 7 2022 12:52PM,250191,22,"NBTY Global, Inc.",,,1
3,Nov 7 2022 12:52PM,250190,22,"NBTY Global, Inc.",,,1
4,Nov 7 2022 12:52PM,250189,22,"NBTY Global, Inc.",,,1
5,Nov 7 2022 12:49PM,250188,10,"Citieffe, Inc.",,,1
6,Nov 7 2022 12:45PM,250187,10,Emerginnova,,,5
7,Nov 7 2022 12:42PM,250186,10,"Snap Medical Industries, LLC",,,18
8,Nov 7 2022 12:32PM,250182,19,"AdvaGen Pharma, Ltd",,,2
9,Nov 7 2022 12:28PM,250181,19,"Innogenix, LLC",,19,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 1:03PM,250193,10,"Citieffe, Inc.",1,,
1,Nov 7 2022 12:57PM,250192,10,ISDIN Corporation,11,,
2,Nov 7 2022 12:52PM,250191,22,"NBTY Global, Inc.",1,,
3,Nov 7 2022 12:52PM,250190,22,"NBTY Global, Inc.",1,,
4,Nov 7 2022 12:52PM,250189,22,"NBTY Global, Inc.",1,,
5,Nov 7 2022 12:49PM,250188,10,"Citieffe, Inc.",1,,
6,Nov 7 2022 12:45PM,250187,10,Emerginnova,5,,
7,Nov 7 2022 12:42PM,250186,10,"Snap Medical Industries, LLC",18,,
8,Nov 7 2022 12:32PM,250182,19,"AdvaGen Pharma, Ltd",2,,
9,Nov 7 2022 12:28PM,250181,19,"Innogenix, LLC",6,19,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 1:03PM,250193,10,"Citieffe, Inc.",1,,
1,Nov 7 2022 12:57PM,250192,10,ISDIN Corporation,11,,
2,Nov 7 2022 12:52PM,250191,22,"NBTY Global, Inc.",1,,
3,Nov 7 2022 12:52PM,250190,22,"NBTY Global, Inc.",1,,
4,Nov 7 2022 12:52PM,250189,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 1:03PM,250193,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Nov 7 2022 12:57PM,250192,10,ISDIN Corporation,11.0,NaN,NaN
2,Nov 7 2022 12:52PM,250191,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Nov 7 2022 12:52PM,250190,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Nov 7 2022 12:52PM,250189,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 7 2022 1:03PM,250193,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Nov 7 2022 12:57PM,250192,10,ISDIN Corporation,11.0,0.0,0.0
2,Nov 7 2022 12:52PM,250191,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Nov 7 2022 12:52PM,250190,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Nov 7 2022 12:52PM,250189,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2501699,51.0,18.0,5.0
15,1000598,23.0,52.0,0.0
16,500251,1.0,1.0,0.0
18,250131,1.0,0.0,0.0
19,1751185,12.0,25.0,0.0
21,1000591,4.0,0.0,0.0
22,750570,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2501699,51.0,18.0,5.0
1,15,1000598,23.0,52.0,0.0
2,16,500251,1.0,1.0,0.0
3,18,250131,1.0,0.0,0.0
4,19,1751185,12.0,25.0,0.0
5,21,1000591,4.0,0.0,0.0
6,22,750570,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,51.0,18.0,5.0
1,15,23.0,52.0,0.0
2,16,1.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,12.0,25.0,0.0
5,21,4.0,0.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,51.0
1,15,Released,23.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21,22
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,18.0,52.0,1.0,0.0,25.0,0.0,0.0
Released,51.0,23.0,1.0,1.0,12.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,52.0,1.0,0.0,25.0,0.0,0.0
2,Released,51.0,23.0,1.0,1.0,12.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,18.0,52.0,1.0,0.0,25.0,0.0,0.0
2,Released,51.0,23.0,1.0,1.0,12.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()